In [1]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import imblearn
import collections

import sklearn
from sklearn.metrics import *
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.neural_network import MLPClassifier

from sklearn.metrics import classification_report,confusion_matrix

from keras.utils import to_categorical

from keras.utils.vis_utils import plot_model

from collections import Counter
from imblearn.under_sampling import OneSidedSelection
from imblearn.over_sampling import ADASYN

import pickle

Load the Data

In [2]:
csv_file = 'actionstatespaceTBharatUMomo_combined.csv'
dataframe = pd.read_csv(csv_file)

In [3]:
dataframe.head()

#print(dataframe["Time Since Last Charge"].mean())
#print(dataframe["Time Since Last Action Formatted"].mean())

,Action,Last Action,Session Time,Time Since Last Charge,Session Time Formatted,Time Since Last Action,Time Since Last Action Formatted,Camera Mood,Gaze,Noise,...,st1,st2,st3,st4,st5,st6,st7,st8,st9,st10
0,7,17,0:01:25.357339,0.00,1.42,0:00:01.000000,0.02,Sad,1,None,...,0,0,01:25.357339,01:25,1,1,1,25,0.42,1.42
1,9,7,0:02:01.922814,0.60,2.02,0:00:36.565475,0.60,Neutral,0,None,...,0,0,02:01.922814,02:01,2,2,2,1,0.02,2.02
2,10,9,0:02:34.035886,1.15,2.57,0:00:32.113072,0.53,Sad,1,Low,...,0,0,02:34.035886,02:34,2,2,2,34,0.57,2.57
3,8,10,0:03:01.126931,1.60,3.02,0:00:27.091045,0.45,Neutral,0,None,...,0,0,03:01.126931,03:01,3,3,3,1,0.02,3.02
4,3,8,0:04:12.233820,2.78,4.20,0:01:11.106889,1.18,Happy,1,None,...,0,0,04:12.233820,04:12,4,4,4,12,0.20,4.20


Format and One-hot encode the data

In [4]:
dataframe.drop("tsla1", axis=1, inplace=True)
dataframe.drop("tsla2", axis=1, inplace=True)
dataframe.drop("tsla3", axis=1, inplace=True)
dataframe.drop("tsla4", axis=1, inplace=True)
dataframe.drop("tsla5", axis=1, inplace=True)
dataframe.drop("tsla6", axis=1, inplace=True)
dataframe.drop("st1", axis=1, inplace=True)
dataframe.drop("st2", axis=1, inplace=True)
dataframe.drop("st3", axis=1, inplace=True)
dataframe.drop("st4", axis=1, inplace=True)
dataframe.drop("st5", axis=1, inplace=True)
dataframe.drop("st6", axis=1, inplace=True)
dataframe.drop("st7", axis=1, inplace=True)
dataframe.drop("st8", axis=1, inplace=True)
dataframe.drop("st9", axis=1, inplace=True)
dataframe.drop("st10", axis=1, inplace=True)
dataframe.drop("Session Time", axis=1, inplace=True)
dataframe.drop("Time Since Last Action", axis=1, inplace=True)

dataframe.drop("Session Time Formatted", axis=1, inplace=True)
#dataframe.drop("Time Since Last Charge", axis=1, inplace=True)  

In [5]:
dataframe.head(100)

,Action,Last Action,Time Since Last Charge,Time Since Last Action Formatted,Camera Mood,Gaze,Noise,Input Mood,Activity
0,7,17,0.00,0.02,Sad,1,None,Neutral,Relaxing
1,9,7,0.60,0.60,Neutral,0,None,Neutral,Relaxing
2,10,9,1.15,0.53,Sad,1,Low,Neutral,Relaxing
3,8,10,1.60,0.45,Neutral,0,None,Neutral,Relaxing
4,3,8,2.78,1.18,Happy,1,None,Neutral,Relaxing
...,...,...,...,...,...,...,...,...,...
95,14,10,3.95,0.40,Sad,1,Low,Neutral,Studying
96,9,14,5.55,1.60,Sad,1,Low,Neutral,Studying
97,9,9,6.60,1.05,Sad,1,Low,Positive,Studying
98,3,9,6.94,0.32,Sad,1,Low,Positive,Studying


In [6]:
pd.set_option('display.max_columns', None)
dataframe = pd.get_dummies(dataframe, columns=["Activity"], prefix=["Activity"])
dataframe = pd.get_dummies(dataframe, columns=["Camera Mood"], prefix=["Camera_Mood"])
dataframe = pd.get_dummies(dataframe, columns=["Input Mood"], prefix=["Input Mood"])
#both = pd.concat([dataframe, dum_df])
#dataframe.drop("Input Mood", axis=1, inplace=True) 
#both.head(5)

dataframe = pd.get_dummies(dataframe, columns=["Noise"], prefix=["Noise"])
dataframe = pd.get_dummies(dataframe, columns=["Last Action"], prefix=["Last Action"])
dataframe.head(100)

,Action,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Relaxing,Activity_Resting,Activity_Studying,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Camera_Mood_Surprise,Input Mood_Neutral,Input Mood_Positive,Noise_High,Noise_Low,Noise_None,Last Action_1,Last Action_2,Last Action_3,Last Action_4,Last Action_5,Last Action_6,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_15,Last Action_16,Last Action_17
0,7,0.00,0.02,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,9,0.60,0.60,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,10,1.15,0.53,1,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,8,1.60,0.45,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,3,2.78,1.18,1,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,14,3.95,0.40,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
96,9,5.55,1.60,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
97,9,6.60,1.05,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
98,3,6.94,0.32,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [7]:
X = dataframe.drop('Action', axis=1)
y = dataframe['Action']

In [8]:
X.head(5)

,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Relaxing,Activity_Resting,Activity_Studying,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Camera_Mood_Surprise,Input Mood_Neutral,Input Mood_Positive,Noise_High,Noise_Low,Noise_None,Last Action_1,Last Action_2,Last Action_3,Last Action_4,Last Action_5,Last Action_6,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_15,Last Action_16,Last Action_17
0,0.00,0.02,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0.60,0.60,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,1.15,0.53,1,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,1.60,0.45,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,2.78,1.18,1,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [9]:
X_train = X[0:175]
X_test = X[175:238]
y_train = y[0:175]
y_test = y[175:238]

In [10]:
X_train

,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Relaxing,Activity_Resting,Activity_Studying,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Camera_Mood_Surprise,Input Mood_Neutral,Input Mood_Positive,Noise_High,Noise_Low,Noise_None,Last Action_1,Last Action_2,Last Action_3,Last Action_4,Last Action_5,Last Action_6,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_15,Last Action_16,Last Action_17
0,0.00,0.02,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0.60,0.60,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,1.15,0.53,1,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,1.60,0.45,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,2.78,1.18,1,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,19.48,0.52,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
171,20.52,1.03,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
172,21.55,1.02,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
173,23.33,1.78,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [11]:
counter = Counter(y_train)
print(counter)

Counter({9: 24, 16: 16, 1: 13, 11: 13, 10: 12, 3: 12, 6: 11, 15: 11, 7: 10, 14: 10, 5: 9, 12: 9, 4: 8, 2: 6, 8: 5, 17: 4, 13: 2})


In [12]:
counter = Counter(y_test)
print(counter)

Counter({16: 11, 9: 8, 1: 6, 14: 5, 11: 5, 7: 4, 12: 4, 6: 4, 10: 3, 15: 3, 3: 3, 5: 2, 4: 2, 2: 1, 8: 1, 17: 1})


In [13]:
X_train_scaled = X_train.copy()

col_names = ['Time Since Last Charge', 'Time Since Last Action Formatted']
features = X_train_scaled[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
X_train_scaled[col_names] = features

filename = 'TBharatUMomo_scaler2.sav'
#pickle.dump(scaler, open(filename, 'wb'))

In [14]:
scaler2 = pickle.load(open(filename, 'rb'))

X_test_scaled = X_test.copy()

features = X_test_scaled[col_names]
features = scaler2.transform(features.values)
X_test_scaled[col_names] = features

X_test_scaled.head()

,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Relaxing,Activity_Resting,Activity_Studying,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Camera_Mood_Surprise,Input Mood_Neutral,Input Mood_Positive,Noise_High,Noise_Low,Noise_None,Last Action_1,Last Action_2,Last Action_3,Last Action_4,Last Action_5,Last Action_6,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_15,Last Action_16,Last Action_17
175,-1.384065,-0.600981,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
176,-1.360829,-0.501422,1,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
177,-1.216250,0.071042,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
178,-1.180105,-0.439197,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
179,-1.137506,-0.414308,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


Load and Train the Model

In [16]:
mlp = MLPClassifier(max_iter=10000, activation='relu', hidden_layer_sizes=(15)) #doesn't converge at 500 so put to higher, do we want it to converge?
mlp

MLPClassifier(hidden_layer_sizes=15, max_iter=10000)

In [17]:
mlp.fit(X_train_scaled,y_train)

MLPClassifier(hidden_layer_sizes=15, max_iter=10000)

In [49]:
#filename = 'TBharatUMomo_model2.sav'
#pickle.dump(mlp, open(filename, 'wb'))

In [18]:
#loaded_model = pickle.load(open(filename, 'rb'))
#pred = loaded_model.predict(X_test_scaled)
pred = mlp.predict(X_test_scaled)
pred

array([ 1,  4, 11, 16, 14,  4,  1,  9,  1,  9,  9, 11, 16,  9,  9,  3, 16,
        9, 15,  6, 16,  2, 16, 16,  9,  6,  7, 16, 16, 17,  7,  3, 17, 12,
        7, 16, 12,  2, 17,  7, 16,  7,  7, 17, 17,  7, 17, 17, 17,  1,  9,
        1,  1,  1,  6,  9, 10,  7,  1, 15,  6,  9, 15])

In [19]:
pred = mlp.predict(X_test_scaled)
pred

array([ 1,  4, 11, 16, 14,  4,  1,  9,  1,  9,  9, 11, 16,  9,  9,  3, 16,
        9, 15,  6, 16,  2, 16, 16,  9,  6,  7, 16, 16, 17,  7,  3, 17, 12,
        7, 16, 12,  2, 17,  7, 16,  7,  7, 17, 17,  7, 17, 17, 17,  1,  9,
        1,  1,  1,  6,  9, 10,  7,  1, 15,  6,  9, 15])

In [20]:
y_test.value_counts()

16    11
9      8
1      6
11     5
14     5
6      4
7      4
12     4
3      3
10     3
15     3
4      2
5      2
2      1
8      1
17     1
Name: Action, dtype: int64

In [21]:
from collections import Counter
Counter(pred)

Counter({1: 8,
         4: 2,
         11: 2,
         16: 10,
         14: 1,
         9: 10,
         3: 2,
         15: 3,
         6: 4,
         2: 2,
         7: 8,
         17: 8,
         12: 2,
         10: 1})